In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd "/content/drive/MyDrive/computer-vision-plant-detection"

In [ ]:
!pip install -r requirements.txt
# !pip list

In [ ]:
# version 1
# from ultralytics import YOLO
# import os

# # Load a pretrained YOLOv8m model

# model = YOLO("yolov10x.pt")

# # Train the model with enhanced augmentation

# results = model.train(
#     data="config.yaml",
#     epochs=100,
#     imgsz=640,
#     batch=4,
# )

# # Evaluate the model

# val_results = model.val()

# # The best model is automatically saved during training

# best_model_path = os.path.join(results.save_dir, "weights", "best.pt")

# print(f"Best model saved as: {best_model_path}")

# # Optionally, save the final model explicitly

# final_model_path = os.path.join(results.save_dir, "weights", "last.pt")

# print(f"Final model saved as: {final_model_path}")

# # Optionally, you can run inference on test images

# model.predict(source="data/images/val/", save=True, conf=0.25)

In [ ]:
# version 2
from ultralytics import YOLO
import os
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import yaml

# Check for GPU availability and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Load a pretrained YOLOv8s model (smaller model for memory efficiency)
model = YOLO("yolov10x.pt").to(device)
# Define data augmentation
transform = transforms.Compose(
    [
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.RandomResizedCrop(640, scale=(0.8, 1.0)),
    ]
)
# Custom dataset class for plant images


class PlantDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dir, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = [
            img
            for img in os.listdir(img_dir)
            if img.endswith((".png", ".jpg", ".jpeg"))
        ]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(
            self.label_dir, self.images[idx].rsplit(".", 1)[0] + ".txt"
        )
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        # Here we're just returning the image path and label path
        # YOLO will handle the actual loading and processing
        return img_path, label_path


# Create custom datasets
train_dataset = PlantDataset(
    "data/images/train/", "data/labels/train/", transform=transform
)
val_dataset = PlantDataset("data/images/val/", "data/labels/val/")
# Train the model with custom settings
results = model.train(
    data="config.yaml",
    epochs=200,
    imgsz=640,
    batch=16,
    optimizer="AdamW",
    lr0=1e-3,
    lrf=1e-4,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,
    cls=0.5,
    dfl=1.5,
    mixup=0.2,
    copy_paste=0.1,
    device=device,
    amp=True,  # Enable mixed precision training
    workers=4,  # Number of worker threads for data loading
    cos_lr=True,  # Use cosine learning rate scheduler
    # resume=True,  # Resume training from last checkpoint
    patience=20,  # Early stopping
    val=True,
    save_period=5,  # Save model every 5 epochs (replaces val_period)
)
# Evaluate the model
val_results = model.val()
# Save the best model
best_model_path = os.path.join(results.save_dir, "weights", "best.pt")
print(f"Best model saved as: {best_model_path}")
# Optionally, save the final model explicitly
final_model_path = os.path.join(results.save_dir, "weights", "last.pt")
print(f"Final model saved as: {final_model_path}")
# Run inference on validation images with confidence threshold
model.predict(source="data/images/val/", save=True, conf=0.25, device=device)
# Export the model to ONNX format for faster inference
model.export(format="onnx", dynamic=True, simplify=True)

In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from ultralytics import YOLO


def load_existing_labels(label_path):
    with open(label_path, "r") as f:
        return [line.strip().split() for line in f.readlines()]


def convert_yolo_to_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = map(float, yolo_bbox)
    x1 = int((x_center - width / 2) * img_width)
    y1 = int((y_center - height / 2) * img_height)
    x2 = int((x_center + width / 2) * img_width)
    y2 = int((y_center + height / 2) * img_height)
    return [x1, y1, x2, y2]


def convert_bbox_to_yolo(bbox, img_width, img_height):
    x1, y1, x2, y2 = bbox
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    x_center = (x1 + x2) / (2 * img_width)
    y_center = (y1 + y2) / (2 * img_height)
    return [x_center, y_center, width, height]


def iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return intersection / (area1 + area2 - intersection)


def process_image(image_path, label_path, model, iou_threshold=0.5):
    print(f"Processing image: {image_path}")
    # Load image
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]
    # Load existing labels
    existing_labels = load_existing_labels(label_path)
    existing_bboxes = [
        convert_yolo_to_bbox(label[1:], img_width, img_height)
        for label in existing_labels
    ]
    # Detect plants using YOLO
    results = model(img)
    new_labels = []
    # Check if there are any detections
    if len(results) > 0 and len(results[0].boxes) > 0:
        for det in results[0].boxes.data:
            x1, y1, x2, y2, conf, cls = det.tolist()
            if cls == 0:  # Assuming 0 is the class for plants
                new_bbox = [int(x1), int(y1), int(x2), int(y2)]
                # Check if the new detection overlaps with existing labels
                is_new = all(
                    iou(new_bbox, existing_bbox) < iou_threshold
                    for existing_bbox in existing_bboxes
                )
                if is_new:
                    yolo_bbox = convert_bbox_to_yolo(new_bbox, img_width, img_height)
                    new_labels.append([0] + yolo_bbox)
                    existing_bboxes.append(
                        new_bbox
                    )  # Add the new bbox to existing_bboxes
    return new_labels


def main():
    # Load pre-trained YOLO model
    model = YOLO("runs/detect/train/weights/best.pt")
    val_images_path = "data/images/val"
    val_labels_path = "data/labels/val"
    for image_file in os.listdir(val_images_path):
        if image_file.lower().endswith((".png", ".jpg", ".jpeg")):
            image_path = os.path.join(val_images_path, image_file)
            label_file = os.path.splitext(image_file)[0] + ".txt"
            label_path = os.path.join(val_labels_path, label_file)
            new_labels = process_image(image_path, label_path, model)
            if new_labels:
                # Append new labels to the existing file
                with open(label_path, "a") as f:
                    for label in new_labels:
                        f.write(" ".join(map(str, label)) + "\n")
                print(
                    f"Processed {image_file}: {len(new_labels)} new plants detected and labeled"
                )
            else:
                print(f"Processed {image_file}: No new plants detected")


main()